In [ ]:
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

files_path = 'imu-dataset-processed\combine'
# Path to folder containing Excel files
folder_path = os.path.join(os.getcwd(), files_path)

# Get a list of all files in the folder with a .xlsx extension
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Shuffle the list of Excel files randomly
random.shuffle(excel_files)

# Split the list of files into training and test sets
train_files, test_files = train_test_split(excel_files, test_size=0.3, random_state=42)  # Adjust test_size as needed

# Initialize empty lists for training and test sets
train_data = pd.DataFrame()
test_data = pd.DataFrame()

# Load the files into the respective sets
for file in train_files:
    df = pd.read_excel(os.path.join(folder_path, file))
    train_data = pd.concat([train_data, df], ignore_index=True)

for file in test_files:
    df = pd.read_excel(os.path.join(folder_path, file))
    test_data = pd.concat([test_data, df], ignore_index=True)

# Separate features (X) and labels (y)
X_train = train_data.drop(columns=['Fall'])  # Assuming 'Fall' is the label column
y_train = train_data['Fall']

X_test = test_data.drop(columns=['Fall'])
y_test = test_data['Fall']

print(f'Train Dataset size: {X_train.shape[0]}')
print(f'Test Dataset size: {X_test.shape[0]}')


In [ ]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training set
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print('\nConfusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(classification_report_str)

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
import tensorflow as tf
import tf2onnx

# Convert the Keras model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model)

# Save the ONNX model to a file
with open("fall_detection_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


In [ ]:
import onnxruntime
import numpy as np

# Prepare sample input data
onnx_model_path = "fall_detection_model.onnx"
onnx_session = onnxruntime.InferenceSession(onnx_model_path)

print(X_test[76])

input_data = X_test[50:100].astype(np.float32)
input_data = input_data.reshape((-1, input_data.shape[-1]))

# Perform inference
output = onnx_session.run(None, {'dense_input': input_data})  # Replace 'dense_input' with the actual input name


# Process output
for i in range(50):
    predicted_class = np.argmax(output[0][i])
    print(f"Predicted class for record {i}: {predicted_class}")
